# SqueezeNet Auto 

In [8]:
from __future__ import absolute_import, division, print_function

import itertools
import glob, os

import matplotlib.pylab as plt
import numpy as np


import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from tensorflow import keras
import cv2

import tensorflow.keras.backend as K
from keras_tuner import HyperParameters
from tensorflow.keras import layers
from keras_tuner import RandomSearch,BayesianOptimization, Hyperband
from tensorflow import keras

import neptunecontrib.monitoring.kerastuner as npt_utils
from keras_tuner import HyperParameters, Objective

from keras_tuner.engine import hypermodel

from model_all import SqueezeNetModel, SqueezeNet11Model,SqueezeNetSEAutoModel
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")



source_dir = '../TsinghuaFEDImages'
data_dir = '../TsinghuaFED_class' # landmarks of each subject_emotion

emotion_class = ['fear','neutral','surprise','anger','content','disgust','sad','happy']

IMG_WIDTH = 224
IMG_HEIGHT = 224
nb_classes = len(emotion_class)
ratio = 1.0
EPOCH = 200

def load_dataset(data_folder, img_dir):
   

  img_list = data_folder+'/train.csv'

  f = open(img_list)
  text = f.read()
  lines = text.split()
  train_img_files = []
  train_labels = []
  for l in lines:
    data = l.split(',')
    train_img_files.append(img_dir+'/'+data[0])
    train_labels.append(int(data[1]))


  img_train_array=[]
  
  img_train_array = []
  for filename,label in zip(train_img_files,train_labels):
      
    image= cv2.imread( filename, cv2.COLOR_BGR2RGB)
    image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
    image=np.array(image)
    image = image.astype('float32')
    image /= 255 
    img_train_array.append(image)
      

  img_list = data_folder+'/test.csv'

  f = open(img_list)
  text = f.read()
  lines = text.split()
  test_img_files = []
  test_labels = []
  for l in lines:
    data = l.split(',')
    test_img_files.append(img_dir+'/'+data[0])
    test_labels.append(int(data[1]))


  img_test_array=[]

  
  for filename,label in zip(test_img_files,test_labels):

    image= cv2.imread( filename, cv2.COLOR_BGR2RGB)
    
    image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
    image=np.array(image)
    image = image.astype('float32')
    image /= 255 
    img_test_array.append(image)


  return img_train_array, train_labels, img_test_array, test_labels
    
def transform_dataset(x_train,y_train,x_test,y_test):

    
    
  x_test=    np.array(x_test)
  x_test.reshape(-1, IMG_HEIGHT, IMG_WIDTH, 3)
  y_test= np.array(y_test)
  x_train = np.array(x_train)
  x_train.reshape(-1, IMG_HEIGHT, IMG_WIDTH, 3)

  y_train = np.array(y_train)
  x_train = np.expand_dims(x_train, -1)
  x_test = np.expand_dims(x_test, -1)
  y_test = keras.utils.to_categorical(y_test,len(emotion_class))
  y_train = keras.utils.to_categorical(y_train,len(emotion_class))
  return x_train,y_train,x_test,y_test


Version:  2.5.0
Eager mode:  True
GPU is available


2021-09-08 20:38:25.637500: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-08 20:38:25.637683: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.77GHz coreCount: 40 deviceMemorySize: 7.76GiB deviceMemoryBandwidth: 417.29GiB/s
2021-09-08 20:38:25.637772: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-08 20:38:25.637925: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-08 20:38:25.638046: I tensorflow/core/

In [2]:
import neptune
neptune.init(project_qualified_name=' ...',api_token='....) # your credentials
  #load data set
x_train, y_train, x_test, y_test =load_dataset(data_dir,source_dir)

x_train, y_train, x_test, y_test = transform_dataset(x_train, y_train, x_test, y_test )



In [3]:

#Squeeze random
hp = HyperParameters()

import neptune


PROJECT = "squeezeAutoRandom_Tsinghua"+str(ratio)
neptune.create_experiment(PROJECT)


# This will override the `learning_rate` parameter with your
# own selection of choices

hp.Choice("learning_rate", values=[ 1e-3, 1e-4])



#fn_name = globals()["build_SqueezeNet_11_fixed"](hp)
# SqueezeNet 

mymodel = SqueezeNetModel (classes=nb_classes, IMG_HEIGHT=IMG_HEIGHT,IMG_WIDTH=IMG_WIDTH)


tuner = RandomSearch(
  #build_SqueezeNet_11_fixed,
  #build_squeezenet_auto_model,
  #build_model,

  hypermodel=mymodel,
  hyperparameters = hp,
  objective=Objective("val_accuracy", direction="max"),
  max_trials=200,
  executions_per_trial=1,
  overwrite=True,
  directory="mytest_tsinghua_dir",
  project_name=PROJECT,
  #distribution_strategy=tf.distribute.MirroredStrategy(),
  logger = npt_utils.NeptuneLogger(),

)
tuner.search_space_summary()


   
 

https://app.neptune.ai/cchantra/keras-tuner/e/KER-93
Search space summary
Default search space size: 5
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001], 'ordered': True}
use_bypass (Boolean)
{'default': False, 'conditions': []}
compression (Fixed)
{'conditions': [], 'value': 1.0}
dropout_rate (Choice)
{'default': 0.0, 'conditions': [], 'values': [0.0, 0.5, 0.8], 'ordered': True}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'sgd'], 'ordered': False}


2021-09-08 20:24:38.876261: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-08 20:24:38.876613: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.77GHz coreCount: 40 deviceMemorySize: 7.76GiB deviceMemoryBandwidth: 417.29GiB/s
2021-09-08 20:24:38.876693: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-08 20:24:38.877006: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-08 20:24:38.877288: I tensorflow/core/

In [4]:

""""lr_finder = LRFinder(min_lr=1e-5, 
                  max_lr=1e-3,
                  steps_per_epoch=np.ceil(EPOCH/batch_size),
                  epochs=3)"""
my_callbacks = [
  tf.keras.callbacks.EarlyStopping('val_accuracy', patience=5),
  #lr_finder,
  #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
  #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

tuner.search(x_train[:int(ratio*len(x_train))], y_train[:int(ratio*len(y_train))], epochs=EPOCH, callbacks = my_callbacks,validation_data=(x_test[:int(ratio*len(x_test))], y_test[:int(ratio*len(y_test))]))

models = tuner.get_best_models(num_models=10)



tuner.results_summary()
npt_utils.log_tuner_info(tuner)


#best_model = tuner.get_best_models(num_models=1)[0]
#best_model.evaluate(x_test, y_test)





Trial 15 Complete [00h 00m 08s]
val_accuracy: 0.07999999821186066

Best val_accuracy So Far: 0.1371428519487381
Total elapsed time: 00h 02m 20s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in mytest_tsinghua_dir/squeezeAutoRandom_Tsinghua1.0
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: False
compression: 1.0
dropout_rate: 0.8
optimizer: adam
Score: 0.1371428519487381
Trial summary
Hyperparameters:
learning_rate: 0.001
use_bypass: True
compression: 1.0
dropout_rate: 0.0
optimizer: adam
Score: 0.1257142871618271
Trial summary
Hyperparameters:
learning_rate: 0.001
use_bypass: True
compression: 1.0
dropout_rate: 0.0
optimizer: sgd
Score: 0.09714286029338837
Trial summary
Hyperparameters:
learning_rate: 0.001
use_bypass: True
compression: 1.0
dropout_rate: 0.5
optimizer: sgd
Score: 0.09142857044935226
Trial summary
Hyperparameters:
learning_rate: 0.001
use_bypass: True
compression: 1.

In [5]:
i = 0
while i < len(models):
    print('model  %d size %d'%(i,hypermodel.maybe_compute_model_size(models[i])))
    i = i+1

model  0 size 739528
model  1 size 739528
model  2 size 739528
model  3 size 739528
model  4 size 739528
model  5 size 739528
model  6 size 739528
model  7 size 739528
model  8 size 739528
model  9 size 739528


In [6]:
len(models)

10

In [9]:
#Squeeze Bayesian
hp = HyperParameters()

import neptune

PROJECT = "squeezeBayesian_Tsinghua"+str(ratio)
neptune.create_experiment(PROJECT)


# This will override the `learning_rate` parameter with your
# own selection of choices

hp.Choice("learning_rate", values=[ 1e-3, 1e-4])



#fn_name = globals()["build_SqueezeNet_11_fixed"](hp)
# SqueezeNet 

mymodel = SqueezeNetModel  (classes=nb_classes,IMG_HEIGHT=IMG_HEIGHT,IMG_WIDTH=IMG_WIDTH)


tuner = BayesianOptimization(
  #build_SqueezeNet_11_fixed,
  #build_squeezenet_auto_model,
  #build_model,

  hypermodel=mymodel,
  hyperparameters = hp,
  objective="val_accuracy",
  max_trials=100,
  executions_per_trial=1,
  overwrite=True,
  directory="mytest_tsinghua_dir",
  project_name= PROJECT,
  #distribution_strategy=tf.distribute.MirroredStrategy(),
  num_initial_points=2,
  alpha=0.0001,
  beta=2.6,
  seed=None,
  tune_new_entries=True,
  allow_new_entries=True,
  logger = npt_utils.NeptuneLogger(),

)
tuner.search_space_summary()


  

https://app.neptune.ai/cchantra/keras-tuner/e/KER-95
Search space summary
Default search space size: 5
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001], 'ordered': True}
use_bypass (Boolean)
{'default': False, 'conditions': []}
compression (Fixed)
{'conditions': [], 'value': 1.0}
dropout_rate (Choice)
{'default': 0.0, 'conditions': [], 'values': [0.0, 0.5, 0.8], 'ordered': True}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'sgd'], 'ordered': False}


In [10]:
""""lr_finder = LRFinder(min_lr=1e-5, 
                  max_lr=1e-3,
                  steps_per_epoch=np.ceil(EPOCH/batch_size),
                  epochs=3)"""
my_callbacks = [
  tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10),
  #lr_finder,
  #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
  #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

tuner.search(x_train[:int(ratio*len(x_train))], y_train[:int(ratio*len(y_train))], epochs=EPOCH, callbacks = my_callbacks,validation_data=(x_test[:int(ratio*len(x_test))], y_test[:int(ratio*len(y_test))]))

models = tuner.get_best_models() #(num_models=10)
tuner.results_summary()
npt_utils.log_tuner_info(tuner)

Trial 100 Complete [00h 00m 15s]
val_accuracy: 0.07999999821186066

Best val_accuracy So Far: 0.1542857140302658
Total elapsed time: 00h 27m 35s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in mytest_tsinghua_dir/squeezeBayesian_Tsinghua1.0
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
learning_rate: 0.001
use_bypass: True
compression: 1.0
dropout_rate: 0.8
optimizer: sgd
Score: 0.1542857140302658
Trial summary
Hyperparameters:
learning_rate: 0.001
use_bypass: True
compression: 1.0
dropout_rate: 0.8
optimizer: sgd
Score: 0.1542857140302658
Trial summary
Hyperparameters:
learning_rate: 0.001
use_bypass: True
compression: 1.0
dropout_rate: 0.8
optimizer: sgd
Score: 0.1542857140302658
Trial summary
Hyperparameters:
learning_rate: 0.001
use_bypass: True
compression: 1.0
dropout_rate: 0.8
optimizer: sgd
Score: 0.1371428519487381
Trial summary
Hyperparameters:
learning_rate: 0.001
use_bypass: True
compression: 1.0
dropo

In [11]:


i = 0
while i < len(models):
    print('model  %d size %d'%(i,hypermodel.maybe_compute_model_size(models[i])))
    i = i+1

model  0 size 739528


In [12]:
 
#SqueezeAuto Hyper
hp   = HyperParameters()


# This will override the `learning_rate` parameter with your
# own selection of choices
hp.Choice("learning_rate", values=[ 1e-3, 1e-4])


import neptune

PROJECT = "squeezeHyperTsinghua"+str(ratio)
neptune.create_experiment(PROJECT)


#fn_name = globals()["build_SqueezeNet_11_fixed"](hp)
# SqueezeNet 

mymodel = SqueezeNetModel(classes=nb_classes,IMG_HEIGHT=IMG_HEIGHT,IMG_WIDTH=IMG_WIDTH)



tuner = Hyperband(
  #build_SqueezeNet_11_fixed,
  #build_squeezenet_auto_model,
  #build_model,

  hypermodel=mymodel,
  hyperparameters = hp,
  objective="val_accuracy",
  #max_trials=20,
  executions_per_trial=1,
  overwrite=True,
  directory="mytest_tsinghua_dir",
  project_name=PROJECT,
  #distribution_strategy=tf.distribute.MirroredStrategy(),
  max_epochs=EPOCH,
  logger = npt_utils.NeptuneLogger(),

)
tuner.search_space_summary()


    


https://app.neptune.ai/cchantra/keras-tuner/e/KER-96
Search space summary
Default search space size: 5
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001], 'ordered': True}
use_bypass (Boolean)
{'default': False, 'conditions': []}
compression (Fixed)
{'conditions': [], 'value': 1.0}
dropout_rate (Choice)
{'default': 0.0, 'conditions': [], 'values': [0.0, 0.5, 0.8], 'ordered': True}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'sgd'], 'ordered': False}


In [13]:

""""lr_finder = LRFinder(min_lr=1e-5, 
                  max_lr=1e-3,
                  steps_per_epoch=np.ceil(EPOCH/batch_size),
                  epochs=3)"""
my_callbacks = [
  tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10),
  #lr_finder,
  #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
  #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

tuner.search(x_train[:int(ratio*len(x_train))], y_train[:int(ratio*len(y_train))], epochs=EPOCH, callbacks = my_callbacks,validation_data=(x_test[:int(ratio*len(x_test))], y_test[:int(ratio*len(y_test))]))

models = tuner.get_best_models() #(num_models=10)
tuner.results_summary()

Trial 24 Complete [00h 00m 05s]
val_accuracy: 0.1257142871618271

Best val_accuracy So Far: 0.1257142871618271
Total elapsed time: 00h 02m 03s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in mytest_tsinghua_dir/squeezeHyperTsinghua1.0
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: True
compression: 1.0
dropout_rate: 0.0
optimizer: sgd
tuner/epochs: 3
tuner/initial_epoch: 0
tuner/bracket: 4
tuner/round: 0
Score: 0.1257142871618271
Trial summary
Hyperparameters:
learning_rate: 0.001
use_bypass: True
compression: 1.0
dropout_rate: 0.5
optimizer: sgd
tuner/epochs: 3
tuner/initial_epoch: 0
tuner/bracket: 4
tuner/round: 0
Score: 0.1257142871618271
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: True
compression: 1.0
dropout_rate: 0.8
optimizer: sgd
tuner/epochs: 3
tuner/initial_epoch: 0
tuner/bracket: 4
tuner/round: 0
Score: 0.11999999731779099
Trial summary
Hyperparamet

In [14]:
i = 0
while i < len(models):
    print('model  %d size %d'%(i,hypermodel.maybe_compute_model_size(models[i])))
    i = i+1

model  0 size 739528
